In [135]:
import tensorflow as tf
tf.reset_default_graph()

In [136]:
#get in the data
chinese = []
english_phoeneme = []
with open('dataset.txt') as f:
    for line in f:
        temp = line.split('\t')
        chinese.append(temp[1])
        english_phoeneme.append(temp[2])
english_phoeneme = [phoeneme.split('\n')[0].split() for phoeneme in english_phoeneme]
chinese = [list(word) for word in chinese]
for ele in chinese:
    if ' ' in ele:
        ele.remove(' ')
def prepare_vocab_input():
    vocab_inputs = []
    with open('./english_phoneme_vocabulary_output.txt') as file:
        for line in file:
            vocab_inputs.append(line.split('\n')[0])
    vocab_inputs.remove('_PAD')
    vocab_inputs.remove('_GO')
    vocab_inputs.remove('_EOS')
    vocab_inputs.remove('_UNK')
    vocab_inputs = ['PAD', 'EOS'] + (vocab_inputs)
    return vocab_inputs
def prepare_vocab_predict():
    vocab_predict = list(chinese_id_dict.keys())
    return vocab_predict
# prepare chinese_id_dict
chinese_id_list = []
with open('./chinese_vocabulary.txt') as file7:
    for line in file7:
        chinese_id_list.append(line.split('\n')[0])
chinese_id_list = ['PAD', 'EOS'] + chinese_id_list
chinese_id_dict = {}
for i in range(len(chinese_id_list)):
    chinese_id_dict[chinese_id_list[i]] = i
# finish preparation for chinese_id_dict
vocab_inputs = prepare_vocab_input()
vocab_predict = prepare_vocab_predict()
english_phoneme_dict = {}
for i in range(len(vocab_inputs)):
    english_phoneme_dict[vocab_inputs[i]] = i


In [304]:
def feeding(inputs, labels):
    inputs_int = []; predict_int = []
    for i in range(len(inputs)):
        single_input = []
        single_predict = []
        tmp_inputs = max([len(x) for x in inputs])
        print(inputs)
        print(tmp_inputs)
        tmp_labels = max([len(x) for x in labels])
        tmp = max(tmp_inputs,tmp_labels)

        for x in range(tmp):
            try:
                single_input += [english_phoneme_dict[inputs[i][x]]]
            except:
                single_input += [0]
        for x in range(tmp):
            try:
                single_predict += [chinese_id_dict[labels[i][x]]]
            except:
                single_input +=[0]
        inputs_int.append(single_input); predict_int.append(single_predict)
        print(inputs_int)
        print(predict_int)
    enc_input, _ = helpers.batch(inputs_int)
    dec_target, _ = helpers.batch([(sequence) + [1] for sequence in predict_int])
    dec_input, _ = helpers.batch([[1] + (sequence) for sequence in inputs_int])
    
    return {encoder_inputs: enc_input, decoder_inputs: dec_input, decoder_targets: dec_target}

In [138]:
greatestvalue_predict = 42
encoder_inputs = tf.placeholder(shape = (None, None), dtype = tf.int32)
decoder_targets = tf.placeholder(shape = (None, None), dtype = tf.int32)
decoder_inputs = tf.placeholder(shape = (None, None), dtype = tf.int32)


encoder_embeddings = tf.Variable(tf.random_uniform([len(vocab_inputs), greatestvalue_predict]
                                                   , -1.0, 1.0), dtype = tf.float32)

decoder_embeddings = tf.Variable(tf.random_uniform([len(vocab_predict), greatestvalue_predict]
                                                   , -1.0, 1.0), dtype = tf.float32)

encoder_inputs_embedded = tf.nn.embedding_lookup(encoder_embeddings, encoder_inputs)
decoder_inputs_embedded = tf.nn.embedding_lookup(decoder_embeddings, decoder_inputs)


### Encoder

In [139]:
hidden = 280

cells = []
for _ in range(2):
    cell = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.GRUCell(hidden), input_keep_prob=0.5)
    cells.append(cell)
multicell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=False)

_, encoder_final_state = tf.nn.dynamic_rnn(multicell, encoder_inputs_embedded,
                                           dtype = tf.float32, time_major = True)


### Decoder

In [140]:
decoder_cells = []

for _ in range(2):
    cell = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.GRUCell(hidden), input_keep_prob=0.5)
    decoder_cells.append(cell)
decoder_multicell = tf.contrib.rnn.MultiRNNCell(decoder_cells, state_is_tuple=False)

# declare a scope for our decoder, later tensorflow will confuse
decoder_outputs, decoder_final_state = tf.nn.dynamic_rnn(decoder_multicell, decoder_inputs_embedded,
                                                             initial_state=encoder_final_state,
                                                             dtype=tf.float32, time_major=True, scope='decoder')

### Prediction

In [141]:
decoder_logits = tf.contrib.layers.linear(decoder_outputs, len(vocab_predict))

decoder_prediction = tf.argmax(decoder_logits, 2)

In [142]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
            labels=tf.one_hot(decoder_targets, depth=len(vocab_predict), dtype=tf.float32),
            logits=decoder_logits)

loss = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(0.003).minimize(loss)


In [145]:
config = tf.ConfigProto(allow_soft_placement = True)
sess = tf.Session(config=config)
saver = tf.train.import_meta_graph('./model/model.ckpt.meta')
saver.restore(sess, "./model/model.ckpt")

INFO:tensorflow:Restoring parameters from ./model/model.ckpt


In [147]:
sess.run(tf.global_variables_initializer())

In [305]:
feed=feeding(english_phoeneme[2: 4], chinese[2: 4])

[['M', 'UH', 'HH', 'AA', 'M', 'AH', 'D', 'K', 'AH', 'L', 'IY', 'F', 'AH', 'AE', 'L', 'K', 'IH', 'N', 'D', 'IY'], ['T', 'S', 'AA', 'G', 'T', 'AA', 'CH', 'AH', 'R', 'AH', 'N', 'L', 'AA', 'HH', 'Y', 'UW', 'AH', 'Z']]
20
[[8, 37, 24, 12, 8, 2, 9, 10, 2, 6, 7, 26, 2, 4, 6, 10, 15, 3, 9, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[775, 634, 1055, 489, 215, 15, 941, 149, 465]]
[['M', 'UH', 'HH', 'AA', 'M', 'AH', 'D', 'K', 'AH', 'L', 'IY', 'F', 'AH', 'AE', 'L', 'K', 'IH', 'N', 'D', 'IY'], ['T', 'S', 'AA', 'G', 'T', 'AA', 'CH', 'AH', 'R', 'AH', 'N', 'L', 'AA', 'HH', 'Y', 'UW', 'AH', 'Z']]
20
[[8, 37, 24, 12, 8, 2, 9, 10, 2, 6, 7, 26, 2, 4, 6, 10, 15, 3, 9, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [13, 11, 12, 21, 13, 12, 32, 2, 5, 2, 3, 6, 12, 24, 29, 22, 2, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[775, 634, 1055, 489, 215, 15, 941, 149, 465], [1439, 728, 52, 353, 214, 997, 1037, 1301, 1446]]


In [280]:
a = [[8, 37, 24, 12, 8, 2, 9, 10, 2, 6, 7, 26, 2, 4, 6, 10, 15, 3, 9, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [13, 11, 12, 21, 13, 12, 32, 2, 5, 2, 3, 6, 12, 24, 29, 22, 2, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [284]:
print(english_phoeneme[2: 4], chinese[2: 4])

[['M', 'UH', 'HH', 'AA', 'M', 'AH', 'D', 'K', 'AH', 'L', 'IY', 'F', 'AH', 'AE', 'L', 'K', 'IH', 'N', 'D', 'IY'], ['T', 'S', 'AA', 'G', 'T', 'AA', 'CH', 'AH', 'R', 'AH', 'N', 'L', 'AA', 'HH', 'Y', 'UW', 'AH', 'Z']] [['穆', '罕', '默', '德', '哈', '利', '法', '金', '迪'], ['朝', '格', '特', '奥', '其', '尔', '洛', '呼', '兹']]


In [298]:
predict = sess.run(decoder_prediction,feed)

In [299]:
predict

array([[1113, 1113],
       [1058, 1113],
       [ 646, 1113],
       [ 646,  409],
       [ 158, 1371],
       [ 158,  534],
       [ 759, 1445],
       [ 600, 1083],
       [ 743, 1083],
       [ 743,  584],
       [ 743, 1083],
       [ 747, 1083],
       [ 926,  304],
       [ 810, 1502],
       [ 723, 1083],
       [1348, 1083],
       [ 135, 1082],
       [ 135, 1082],
       [1246, 1502],
       [1246,  884],
       [1246,  884],
       [1246,  884],
       [1207,  884],
       [ 920,  884],
       [ 253,  253],
       [ 253,  952],
       [ 253,  594],
       [ 253,  253],
       [ 253,  253],
       [ 594,  253],
       [ 594, 1269],
       [ 594,  240]])

In [203]:
english_phoeneme[1: 2], chinese[1: 2]

([['AE', 'K', 'AH', 'K', 'AY', 'JH']], [['阿', '卡', '基', '耶']])

In [266]:
feed

{<tf.Tensor 'Placeholder:0' shape=(?, ?) dtype=int32>: array([[ 8, 13],
        [37, 11],
        [24, 12],
        [12, 21],
        [ 8, 13],
        [ 2, 12],
        [ 9, 32],
        [10,  2],
        [ 2,  5]], dtype=int32),
 <tf.Tensor 'Placeholder_2:0' shape=(?, ?) dtype=int32>: array([[ 1,  1],
        [ 8, 13],
        [37, 11],
        [24, 12],
        [12, 21],
        [ 8, 13],
        [ 2, 12],
        [ 9, 32],
        [10,  2],
        [ 2,  5]], dtype=int32),
 <tf.Tensor 'Placeholder_1:0' shape=(?, ?) dtype=int32>: array([[ 775, 1439],
        [ 634,  728],
        [1055,   52],
        [ 489,  353],
        [ 215,  214],
        [  15,  997],
        [ 941, 1037],
        [ 149, 1301],
        [ 465, 1446],
        [   1,    1]], dtype=int32)}

In [175]:
def label_to_chinese(label):
    chinese = ''
    for i in range(len(label)):
        if label[i][0] == 0 or label[i][0] == 1:
            continue
        chinese += vocab_predict[label[i][0]] + ' '
    return chinese

In [233]:
import numpy as np
for i in range(10):
    rand = np.random.randint(len(english_phoeneme))
    test = feeding(english_phoeneme[rand: rand+1], chinese[rand: rand+1])
    predict = sess.run(decoder_prediction, test)
    print('input: ' + str(english_phoeneme[rand: rand+1]))
    print('supposed label: ' + str(chinese[rand: rand+1]))
    print('predict label:' + str(label_to_chinese(predict))+'\n')

[[2, 6, 17, 8, 7, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[872, 1037, 1342, 797, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
input: [['AH', 'L', 'OW', 'M', 'IY', 'AH']]
supposed label: [['阿', '洛', '米', '亚']]
predict label:筱 长 长 吴 吴 婉 婉 荻 尔 溏 蕾 溏 溏 溏 杰 杰 杰 白 蕾 腊 白 腊 腊 础 炎 腊 柄 柄 杰 仓 仓 仓 平 蕾 蕾 蕾 蕾 挽 挽 溏 溏 挽 

[[18, 6, 7, 2, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[872, 15, 353, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
input: [['EY', 'L', 'IY', 'AH', 'T']]
supposed label: [['阿', '利', '奥']]
predict label:长 芭 芭 芭 芭 芭 芭 芭 寿 复 杰 杰 腊 腊 腊 腊 腊 腊 仙 溏 溏 复 腊 溏 溏 溏 溏 溏 溏 溏 溏 挽 腊 蕾 蕾 蕾 蕾 复 蕾 蕾 蕾 蕾 

[[4, 6, 10, 17, 14, 2, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [200]:
rand

27439

In [196]:
test = feeding(english_phoeneme[2: 3], chinese[2: 3])
predict = sess.run(decoder_prediction, test)
print(test)
print('input: ' + str(english_phoeneme[2: 3]))
print('supposed label: ' + str(chinese[2: 3]))
print('predict label:' + str(label_to_chinese(predict)))

{<tf.Tensor 'Placeholder:0' shape=(?, ?) dtype=int32>: array([[ 8],
       [37],
       [24],
       [12],
       [ 8],
       [ 2],
       [ 9],
       [10],
       [ 2]], dtype=int32), <tf.Tensor 'Placeholder_2:0' shape=(?, ?) dtype=int32>: array([[ 1],
       [ 8],
       [37],
       [24],
       [12],
       [ 8],
       [ 2],
       [ 9],
       [10],
       [ 2]], dtype=int32)}
input: [['M', 'UH', 'HH', 'AA', 'M', 'AH', 'D', 'K', 'AH', 'L', 'IY', 'F', 'AH', 'AE', 'L', 'K', 'IH', 'N', 'D', 'IY']]
supposed label: [['穆', '罕', '默', '德', '哈', '利', '法', '金', '迪']]
predict label:迅 迅 顿 棉 棉 贴 棉 塔 塔 归 
